In [1]:
import glob
import os
from pathlib import Path


pool_keyword = "first"
data_dir = "/cmnfs/proj/prosit/Transformer/Final_Meta_Data"
meta_data_filepath = glob.glob(os.path.join(data_dir, "*meta_data*.parquet"))[0]
meta_data_filepath

'/cmnfs/proj/prosit/Transformer/Final_Meta_Data/Thermo_SRM_Pool_meta_data.parquet'

In [2]:
meta_data_filepath = '/cmnfs/proj/prosit/Transformer/Final_Meta_Data/TUM_first_pool_meta_data.parquet'

In [3]:
import pandas as pd
df = pd.read_parquet(meta_data_filepath)

In [6]:
df = df[df.precursor_charge <= 6]

In [7]:
df.reset_index(drop=True).to_parquet('./TUM_first_pool_meta_data_filtered.parquet', index=False)

In [8]:
pool_folder_path = "/cmnfs/proj/prosit/Transformer/TUM_first_pool_1"
annotations_filepaths = glob.glob(os.path.join(pool_folder_path, "*.parquet"))
annotations_names = [Path(f).stem for f in annotations_filepaths]

annotations_names, annotations_filepaths

(['TUM_first_pool_10_01_01_annotation',
  'TUM_first_pool_11_01_01_annotation',
  'TUM_first_pool_12_01_01_annotation',
  'TUM_first_pool_13_01_01_annotation',
  'TUM_first_pool_14_01_01_annotation',
  'TUM_first_pool_15_01_01_annotation',
  'TUM_first_pool_16_01_01_annotation',
  'TUM_first_pool_17_01_01_annotation',
  'TUM_first_pool_18_01_01_annotation',
  'TUM_first_pool_19_01_01_annotation',
  'TUM_first_pool_1_01_01_annotation',
  'TUM_first_pool_20_01_01_annotation',
  'TUM_first_pool_21_01_01_annotation',
  'TUM_first_pool_22_01_01_annotation',
  'TUM_first_pool_23_01_01_annotation',
  'TUM_first_pool_24_01_01_annotation',
  'TUM_first_pool_25_01_01_annotation',
  'TUM_first_pool_26_01_01_annotation',
  'TUM_first_pool_27_01_01_annotation',
  'TUM_first_pool_28_01_01_annotation',
  'TUM_first_pool_29_01_01_annotation',
  'TUM_first_pool_2_01_01_annotation',
  'TUM_first_pool_30_01_01_annotation',
  'TUM_first_pool_31_01_01_annotation',
  'TUM_first_pool_32_01_01_annotation',
  

In [9]:
input_data_dict = {
    "metadata": "./TUM_first_pool_meta_data_filtered.parquet",
    "annotations": {
        pool_keyword: dict(zip(annotations_names, annotations_filepaths))
    },
    "parameters": {
        "target_column_key": "intensities_raw"
    }
}

input_data_dict

{'metadata': './TUM_first_pool_meta_data_filtered.parquet',
 'annotations': {'first': {'TUM_first_pool_10_01_01_annotation': '/cmnfs/proj/prosit/Transformer/TUM_first_pool_1/TUM_first_pool_10_01_01_annotation.parquet',
   'TUM_first_pool_11_01_01_annotation': '/cmnfs/proj/prosit/Transformer/TUM_first_pool_1/TUM_first_pool_11_01_01_annotation.parquet',
   'TUM_first_pool_12_01_01_annotation': '/cmnfs/proj/prosit/Transformer/TUM_first_pool_1/TUM_first_pool_12_01_01_annotation.parquet',
   'TUM_first_pool_13_01_01_annotation': '/cmnfs/proj/prosit/Transformer/TUM_first_pool_1/TUM_first_pool_13_01_01_annotation.parquet',
   'TUM_first_pool_14_01_01_annotation': '/cmnfs/proj/prosit/Transformer/TUM_first_pool_1/TUM_first_pool_14_01_01_annotation.parquet',
   'TUM_first_pool_15_01_01_annotation': '/cmnfs/proj/prosit/Transformer/TUM_first_pool_1/TUM_first_pool_15_01_01_annotation.parquet',
   'TUM_first_pool_16_01_01_annotation': '/cmnfs/proj/prosit/Transformer/TUM_first_pool_1/TUM_first_pool_1

In [10]:
# later we can feed the dict directly as a data source, for now we stick to json format

import json
with open("input_config.json", 'w') as fp:
    json.dump(input_data_dict, fp)

In [11]:
from prosit_t.data import IntensityDataset
from dlomix.data.feature_extractors import (
    ModificationGainFeature,
    ModificationLocationFeature,
    ModificationLossFeature,
)

BATCH_SIZE = 128
SEQ_LENGTH = 30

int_data = IntensityDataset(
    data_source="input_config.json",
    seq_length=SEQ_LENGTH,
    batch_size=BATCH_SIZE,
    val_ratio=0.15,
    precursor_charge_col="precursor_charge_onehot",
    sequence_col="modified_sequence",
    collision_energy_col="collision_energy_aligned_normed",
    intensities_col="intensities_raw",
    features_to_extract=[
        ModificationLocationFeature(),
        ModificationLossFeature(),
        ModificationGainFeature(),
    ],
    parser="proforma",
)

2023-06-22 09:22:42.878369: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-22 09:22:42.984915: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-06-22 09:22:43.453020: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-06-22 09:22:43.453092: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

KeyError: 1932

In [7]:
"Training examples", BATCH_SIZE * len(int_data.train_data)


NameError: name 'int_data' is not defined

In [ ]:
"Validation examples", BATCH_SIZE * len(int_data.val_data)


In [ ]:
import tensorflow as tf
from dlomix.models import PrositIntensityPredictor
from dlomix.losses import masked_spectral_distance

model = PrositIntensityPredictor(seq_length=30, use_ptm_counts=True)

# create the optimizer object
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

# compile the model  with the optimizer and the metrics we want to use, we can add our custom metric
model.compile(optimizer=optimizer, loss=masked_spectral_distance, metrics=["mae"])


In [ ]:

history = model.fit(int_data.train_data,
                    validation_data=int_data.val_data,
                    epochs=2)